In [2]:
#set up autoreload
%load_ext autoreload
%autoreload 2

In [3]:
import polars as pl
from pathlib import Path

import importlib  
fa_etl_dev = importlib.import_module("fa-etl-dev")

initial data quality checks

1. Check if all parcels in annual file exist in tax & val hist:
    * 36061: yes
    * 42101: not all (606/582083 of props in annual are not in tax hist, all in value hist)
    * 48203: yes

In [68]:
county = "36061"

In [69]:
taxhist = pl.read_parquet(f"/Users/claireboyd/internships/mansueto/fa-etl/dev-data/raw_{county}/parquet/TaxHist{county}.parquet")
valhist = pl.read_parquet(f"/Users/claireboyd/internships/mansueto/fa-etl/dev-data/raw_{county}/parquet/ValHist{county}.parquet")
annual = pl.read_parquet(f"/Users/claireboyd/internships/mansueto/fa-etl/dev-data/raw_{county}/parquet/Prop{county}.parquet")

annual = annual.with_columns([
    pl.col("PropertyID").is_in(taxhist['PropertyID']).alias("in_taxhist"),
    pl.col("PropertyID").is_in(valhist['PropertyID']).alias("in_valhist")
    ])

In [55]:
annual["in_taxhist"].value_counts()

in_taxhist,count
bool,u32
true,582083
false,606


In [56]:
annual["in_valhist"].value_counts()

in_valhist,count
bool,u32
true,582689


In [58]:
annual.filter([
    pl.col("in_taxhist") == False
])
    

PropertyID,PropertyClassID,FATimeStamp,SitusLatitude,SitusLongitude,SitusFullStreetAddress,SitusCity,SitusState,SitusZIP5,FIPS,SitusCensusTract,SitusCensusBlock,SitusGeoStatusCode,FIPS_SitusCensusTract,in_taxhist,in_valhist
i64,str,date,f64,f64,str,str,str,str,str,str,str,str,str,bool,bool
174478576,"""R""",2024-01-19,39.968017,-75.140509,"""1115 N AMERICA…","""PHILADELPHIA""","""PA""","""19123""","""42101""","""014201""","""2019""","""B""","""42101_14201""",false,true
174478665,"""R""",2024-01-19,39.982104,-75.148186,"""815 W NORRIS S…","""PHILADELPHIA""","""PA""","""19122""","""42101""","""037700""","""2016""","""B""","""42101_37700""",false,true
174478690,"""R""",2024-01-19,39.943665,-75.173498,"""701 S 19TH ST …","""PHILADELPHIA""","""PA""","""19146""","""42101""","""001400""","""1004""","""B""","""42101_1400""",false,true
174478776,"""R""",2024-01-19,39.968017,-75.140509,"""1115 N AMERICA…","""PHILADELPHIA""","""PA""","""19123""","""42101""","""014201""","""2019""","""B""","""42101_14201""",false,true
174478783,"""R""",2024-01-19,39.968017,-75.140509,"""1115 N AMERICA…","""PHILADELPHIA""","""PA""","""19123""","""42101""","""014201""","""2019""","""B""","""42101_14201""",false,true
174478851,"""R""",2024-01-19,39.968017,-75.140509,"""1115 N AMERICA…","""PHILADELPHIA""","""PA""","""19123""","""42101""","""014201""","""2019""","""B""","""42101_14201""",false,true
174478865,"""R""",2024-01-19,39.97202,-75.16775,"""924 N 19TH ST""","""PHILADELPHIA""","""PA""","""19130""","""42101""","""013900""","""3006""","""B""","""42101_13900""",false,true
174478633,"""R""",2024-01-19,39.98217,-75.142657,"""2009 N REESE S…","""PHILADELPHIA""","""PA""","""19122""","""42101""","""015600""","""2005""","""9""","""42101_15600""",false,true
174478683,"""R""",2024-01-19,39.983137,-75.146981,"""2053 N 8TH ST …","""PHILADELPHIA""","""PA""","""19122""","""42101""","""037700""","""2014""","""B""","""42101_37700""",false,true


data quality checks:
* check for composition of valhist
* why am i losing data when i merge valhist with prop? 2871980 to 2868382

In [15]:
fa_etl_dev.convert_taxhist("TaxHist36061.txt.zip", "/Users/claireboyd/internships/mansueto/fa-etl/dev-data/raw_small")
fa_etl_dev.convert_valhist("ValHist36061.txt.zip", "/Users/claireboyd/internships/mansueto/fa-etl/dev-data/raw_small")

In [77]:
valhist.filter(
    ((pl.col("AssdTotalValue") == 0) & ((pl.col("MarketValueYear").is_not_null()) | (pl.col("ApprYear").is_not_null())))
)

PropertyID,AssdTotalValue,AssdYear,MarketTotalValue,MarketValueYear,ApprTotalValue,ApprYear,TaxableYear
i64,i64,i64,i64,i64,i64,i64,i64
91847126,0,2022,11488000,2022,0,null,null
91846931,0,2022,9018000,2022,0,null,null
91846943,0,2022,11073000,2022,0,null,null
91847071,0,2022,7274000,2022,0,null,null
91846533,0,2022,11421000,2022,0,null,null
91846901,0,2022,15977000,2022,0,null,null
91847954,0,2022,10707000,2022,0,null,null
91848631,0,2022,9228000,2022,0,null,null
91848390,0,2022,15584000,2022,0,null,null


In [71]:
valhist.shape

(2871980, 8)

In [6]:
taxhist.schema

OrderedDict([('PropertyID', Int64), ('TaxYear', Int64), ('TaxAmt', Int64)])

In [78]:
fa_etl_dev.join(input_dir="/Users/claireboyd/internships/mansueto/fa-etl/dev-data/raw_small",
                valhist_filename="ValHist36061.parquet",
                prop_filename="Prop36061.parquet",
                ranked_deed_filename="ranked_Deed36061.parquet")

FileNotFoundError: No such file or directory (os error 2): ...s/claireboyd/internships/mansueto/fa-etl/dev-data/raw_small/parquet/ValHist36061.parquet

In [21]:
merged = pl.read_parquet("/Users/claireboyd/internships/mansueto/fa-etl/dev-data/raw_small/parquet/merged.parquet")

In [17]:
valhist.shape

# property file - propertyid is not null, data type issues

(2871980, 8)

In [22]:
merged.shape

(2868382, 21)

In [14]:
subset_df = valhist.filter(pl.col("PropertyID") == 0)
subset_df.shape

(0, 8)